# 모델 저장과 복원

In [1]:
# 필요한 라이브러리 설치
!pip install -q h5py pyyaml

In [1]:
# 예제 데이터셋 다운로드
# MNIST 데이터셋으로 모델을 훈련해 가중치 저장하는 예제

from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.1.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델 정의

In [3]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
  ])
    
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기

- 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있음

### 체크포인트 콜백 사용하기

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 896/1000 [=========================>....] - ETA: 0s - loss: 1.1796 - accuracy: 0.6786
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1235 - accuracy: 0.6910 - val_loss: 0.6796 - val_accuracy: 0.8050
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4246 - accuracy: 0.8788
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 443us/sample - loss: 0.4195 - accuracy: 0.8840 - val_loss: 0.5233 - val_accuracy: 0.8390
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2774 - accuracy: 0.9279
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 378us/sample - loss: 0.2729 - accuracy: 0.9290 - val_loss: 0.5004 - val_accuracy: 0.8380
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2012 - accuracy: 0.9552
Epoch 0

In [5]:
!dir {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: E63B-F2B5

 C:\DLstudy\training_1 디렉터리

2020-05-11  오후 10:21    <DIR>          .
2020-05-11  오후 10:21    <DIR>          ..
2020-05-11  오후 10:21                71 checkpoint
2020-05-11  오후 10:21         4,886,700 cp.ckpt.data-00000-of-00001
2020-05-11  오후 10:21             1,222 cp.ckpt.index
               3개 파일           4,887,993 바이트
               2개 디렉터리  178,225,467,392 바이트 남음


- 훈련되지 않은 새로운 모델 만들기
- 가중치만 복원할 때는 원본 모델과 동일한 구조로 모델을 만들어야 함
- 다른 객체라도 구조가 같으면 가중치를 공유할 수 있음

In [6]:
model=create_model()
loss, acc=model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.3629 - accuracy: 0.1060
훈련되지 않은 모델의 정확도: 10.60%


In [7]:
# 체크포인트에서 가중치를 로드하고 다시 평가

model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.3900 - accuracy: 0.8770
복원된 모델의 정확도: 87.70%


### 체크포인트 콜백 매개변수

- 체크포인트 이름을 설정하거나 체크포인트 주기를 조정할 수 있음
- 새로운 모델을 훈련하고 5번 에폭마다 고유한 이름으로 체크포인트 저장하기

In [8]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
# 만들어진 체크포인트 확인
!dir {checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: E63B-F2B5

 C:\DLstudy\training_2 디렉터리

2020-05-11  오후 10:22    <DIR>          .
2020-05-11  오후 10:22    <DIR>          ..
2020-05-11  오후 10:22                81 checkpoint
2020-05-11  오후 10:22         1,628,741 cp-0000.ckpt.data-00000-of-00001
2020-05-11  오후 10:22               402 cp-0000.ckpt.index
2020-05-11  오후 10:19         4,886,700 cp-0001.ckpt.data-00000-of-00001
2020-05-11  오후 10:19             1,222 cp-0001.ckpt.index
2020-05-11  오후 10:19         4,886,700 cp-0002.ckpt.data-00000-of-00001
2020-05-11  오후 10:19             1,222 cp-0002.ckpt.index
2020-05-11  오후 10:19         4,886,700 cp-0003.ckpt.data-00000-of-00001
2020-05-11  오후 10:19             1,222 cp-0003.ckpt.index
2020-05-11  오후 10:19         4,886,700 cp-0004.ckpt.data-00000-of-00001
2020-05-11  오후 10:19             1,222 cp-0004.ckpt.index
2020-05-11  오후 10:22         4,886,700 cp-0005.ckpt.data-00000-of-00001
2020-05-11  오후 10:22             1,222 cp-0005.ckpt.index
2020-05-11  

In [10]:
# 가장 마지막 체크포인트 선택
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [11]:
# 모델을 초기화하고 최근 체크포인트 로드해 테스트

model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4860 - accuracy: 0.8720
복원된 모델의 정확도: 87.20%


### 수동으로 가중치 저장하기

In [12]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4860 - accuracy: 0.8720
복원된 모델의 정확도: 87.20%


## 모델 전체를 저장하기

### HDF5 파일로 저장하기

In [13]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 855us/sample - loss: 1.1882 - accuracy: 0.6380
Epoch 2/5
1000/1000 [==============================] - 0s 210us/sample - loss: 0.4267 - accuracy: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 197us/sample - loss: 0.2815 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 207us/sample - loss: 0.2106 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 192us/sample - loss: 0.1531 - accuracy: 0.9680


In [14]:
# 위의 파일로 모델 만들기
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다

new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 정확도 확인

loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4115 - accuracy: 0.8630
복원된 모델의 정확도: 86.30%


### saved_model을 사용하기

In [23]:
# 새로운 모델 만들기

model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model\my_model') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 816us/sample - loss: 1.2384 - accuracy: 0.6390
Epoch 2/5
1000/1000 [==============================] - 0s 188us/sample - loss: 0.4462 - accuracy: 0.8790
Epoch 3/5
1000/1000 [==============================] - 0s 198us/sample - loss: 0.3066 - accuracy: 0.9210
Epoch 4/5
1000/1000 [==============================] - 0s 198us/sample - loss: 0.2153 - accuracy: 0.9460
Epoch 5/5
1000/1000 [==============================] - 0s 206us/sample - loss: 0.1628 - accuracy: 0.9660
INFO:tensorflow:Assets written to: saved_model\my_model\assets


In [18]:
# saved_model을 만들어 타임스탬프를 이름으로 가진 디렉토리에 저장

import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.experimental' has no attribute 'export_saved_model'

In [24]:
# my_model 경로
!dir saved_model

# assets folder, saved_model.pb, variable folder 들어있음
!dir saved_model\my_model

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: E63B-F2B5

 C:\DLstudy\saved_model 디렉터리

2020-05-11  오후 10:31    <DIR>          .
2020-05-11  오후 10:31    <DIR>          ..
2020-05-11  오후 10:32    <DIR>          my_model
               0개 파일                   0 바이트
               3개 디렉터리  178,206,339,072 바이트 남음
 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: E63B-F2B5

 C:\DLstudy\saved_model\my_model 디렉터리

2020-05-11  오후 10:32    <DIR>          .
2020-05-11  오후 10:32    <DIR>          ..
2020-05-11  오후 10:31    <DIR>          assets
2020-05-11  오후 10:32            83,502 saved_model.pb
2020-05-11  오후 10:31    <DIR>          variables
               1개 파일              83,502 바이트
               4개 디렉터리  178,206,339,072 바이트 남음


In [25]:
# saved model에서 다시 모델 로드

new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.4328 - accuracy: 0.8560
Restored model, accuracy: 85.60%
(1000, 10)
